In [59]:
import pandas as pd
import pathlib
import json

In [84]:
#path_manual = "/export/usuarios_ml4ds/lbartolome/Repos/repos_con_carlos/RAG_tool/data/eval_eqs.xlsx"
path_manual = "/export/usuarios_ml4ds/lbartolome/Repos/repos_con_carlos/RAG_tool/data/eval_eqs_scholar.xlsx"
#path_detected = "/export/usuarios_ml4ds/lbartolome/Repos/repos_con_carlos/RAG_tool/data/eval_eqs_pred.json"
path_detected = "/export/usuarios_ml4ds/lbartolome/Repos/repos_con_carlos/RAG_tool/data/eval_eqs_pred_scholar.json"

In [85]:
df_manual = pd.read_excel(path_manual, header=None)
df_manual.columns = ["from", "to"]
df_manual

,from,to
0,gradients,gradient
1,increasing,increase
2,increased,increase
3,augmentation,/
4,dimension,dimensions
...,...,...
362,processor,processors
363,disease,diseases
364,boosting,boosted
365,enhanced,enhance


In [86]:
equivalent = {}
with pathlib.Path(path_detected).open('r', encoding='utf8') as fin:
    newEq = json.load(fin)['wordlist']
newEq = [x.split(':') for x in newEq]
newEq_from = [x[0] for x in newEq if len(x) == 2]
newEq_to = [x[1] for x in newEq if len(x) == 2]
df_pred = pd.DataFrame(
    {
        "from": newEq_from,
        "to": newEq_to
    }
)
df_pred.iloc[0:302]

,from,to
0,gradients,gradient
1,community,communities
2,dimension,dimensions
3,load,loading
4,loads,loading
...,...,...
297,processor,processors
298,disease,diseases
299,boosted,boosting
300,enhance,enhanced


In [87]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

df_merged = df_manual.merge(df_pred, on='from', how='left', suffixes=('_manual', '_pred'))

true_mappings = []
predicted_mappings = []

for index, row in df_merged.iterrows():
    if row['to_manual'] == '/':  # Word should not be mapped
        true_mappings.append(0)  # 0 means no mapping is correct
        if pd.isna(row['to_pred']):  # If there's no prediction, it's correct
            predicted_mappings.append(0)
        else:
            predicted_mappings.append(1)  # Incorrectly predicted a mapping
    else:  # Word should be mapped
        true_mappings.append(1)  # 1 means mapping is required
        if pd.isna(row['to_pred']):  # No mapping was predicted
            predicted_mappings.append(0)
        elif row['to_manual'] == row['to_pred']:  # Correct mapping
            predicted_mappings.append(1)
        else:
            predicted_mappings.append(0)  # Incorrect mapping

precision = precision_score(true_mappings, predicted_mappings)
recall = recall_score(true_mappings, predicted_mappings)
f1 = f1_score(true_mappings, predicted_mappings)

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

Precision: 0.992
Recall: 0.782
F1 Score: 0.875


In [56]:
print(f1_score(true_mappings, predicted_mappings))

0.8565488565488565


In [90]:
len(df_merged)

368

In [91]:
df_merged = df_manual.merge(df_pred, on='from', how='left', suffixes=('_manual', '_pred'))

# Filter rows where there is a valid mapping in the manual dataset (not '/')
# and either the predicted mapping is missing or incorrect
false_negatives = df_merged[
    (df_merged['to_manual'] != '/') &  # Valid mapping exists in manual annotations
    ((df_merged['to_pred'].isna()) | (df_merged['to_manual'] != df_merged['to_pred']))  # No or incorrect prediction
]

false_negatives[60:]

,from,to_manual,to_pred
304,assessing,assess,assessed
308,reservoir,resevoirs,reservoirs
316,dialoguir,dialogue,NaN
317,dialogues,dialogue,dialoguir
325,markov_chains,markov_chain,NaN
326,category,categores,categories
331,synthesized,synthesis,NaN
356,circuit,cirtcuits,circuits
361,film,movie,NaN
365,boosting,boosted,NaN


In [92]:
# Identify cases where the "from" value in df_manual is present in the "to" column of df_pred
reverse_mappings = df_manual[df_manual['from'].isin(df_pred['to'])]
reverse_mappings

,from,to
31,recommender_system,recommendation_system
40,identifying,indentify
52,operation,operations
87,emotions,emotion
125,includes,include
148,ecological,/
150,offered,offer
184,summaries,summarization
219,contrastive,/
224,agricultural,agriculture


In [105]:
#df_valid = pd.read_excel("/export/usuarios_ml4ds/lbartolome/Repos/repos_con_carlos/RAG_tool/data/cpv45_equivalences_test_annotated.xlsx")
df_valid = pd.read_excel("/export/usuarios_ml4ds/lbartolome/Repos/repos_con_carlos/RAG_tool/data/scholar_equivalences_test_valid.xlsx").iloc[0:254]
df_valid["related_model"] = df_valid["equivalence"].apply(lambda x : 1 if x != '[]' else 0)
df_valid[df_valid["related_model"]==1]

,Unnamed: 0,origin,equivalence,related,related_model
0,0,"['gradient', 'gradients']",[{'gradient': ['gradients']}],1,1
2,2,"['communities', 'community']",[{'communities': ['community']}],1,1
3,3,"['dimensions', 'dimension']",[{'dimensions': ['dimension']}],1,1
4,4,"['loading', 'load', 'loads']","[{'loading': ['load', 'loads']}]",1,1
7,7,"['sentences', 'sentence']",[{'sentence': ['sentences']}],1,1
...,...,...,...,...,...
248,248,"['uncertain', 'uncertainties', 'uncertainty']","[{'uncertainty': ['uncertain', 'uncertainties']}]",1,1
250,250,"['transformer', 'transformers']",[{'transformer': ['transformers']}],1,1
251,251,"['processors', 'processor']",[{'processors': ['processor']}],1,1
252,252,"['diseases', 'disease']",[{'diseases': ['disease']}],1,1


In [103]:
model_clusters = df_valid["related_model"].values
user_evaluation =  df_valid["related"].values

In [106]:
precision = precision_score(user_evaluation, model_clusters)
recall = recall_score(user_evaluation, model_clusters)
f1 = f1_score(user_evaluation, model_clusters)

print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')

Precision: 0.995
Recall: 0.835
F1 Score: 0.908


In [113]:
### getting lenght of wordlists (eqs)

path_detected = [
    "/export/usuarios_ml4ds/lbartolome/Repos/repos_con_carlos/RAG_tool/data/scholar/optimized/3.equivalence_detection/equivalences_lst/S2_Kwds3_AI_with_text_30000_both_1.json",
    "/export/usuarios_ml4ds/lbartolome/Repos/repos_con_carlos/RAG_tool/data/scholar/optimized/3.equivalence_detection/equivalences_lst/S2_Kwds3_AI_with_text_30000_both_2.json",
    "/export/usuarios_ml4ds/lbartolome/Repos/repos_con_carlos/RAG_tool/data/scholar/optimized/3.equivalence_detection/equivalences_lst/S2_Kwds3_AI_with_text_30000_both_3.json",
    #"/export/usuarios_ml4ds/lbartolome/Repos/repos_con_carlos/RAG_tool/data/scholar/optimized/3.equivalence_detection/equivalences_lst/manual.json"
]

all_eqs = []
for el in path_detected:
    with pathlib.Path(el).open('r', encoding='utf8') as fin:
        newEq = json.load(fin)['wordlist']
        all_eqs += newEq
len(all_eqs)

133